# BTC Realized Volatility

In [98]:
import ccxt
import pandas as pd
import numpy as np
import arch
import datetime as dt
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import plotly.express as px

In [129]:
# Get Bitcoin daily OHLC data 

# from ccxt binance
# Define parameters
symbol = 'BTC/USDT'
timeframe = '1d'
limit = 1000

# Get OHLC data from Binance
ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=limit)
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# from coingecko
# resp = requests.get('https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=365')
# data = resp.json()['prices']
# df = pd.DataFrame(data, columns=['timestamp', 'close'])
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# df.set_index('timestamp', inplace=True)

# df = pd.read_csv('BTC_daily.csv', parse_dates=['Date'], index_col='Date')

# df.columns = [x.lower() for x in df.columns]

In [130]:
df.head()


,open,high,low,close,volume
timestamp,,,,,
2020-08-18,12281.15,12387.77,11817.93,11945.01,75923.835527
2020-08-19,11945.10,12020.08,11561.00,11754.59,73940.169606
2020-08-20,11754.38,11888.00,11668.00,11853.55,46085.254351
2020-08-21,11853.54,11878.00,11485.81,11531.34,64448.306142
2020-08-22,11531.23,11686.00,11376.81,11662.96,43678.701646


In [131]:
window_size = 30 # in days

# Compute annualized realized volatility rolling window using Close to close
df['log_return'] = np.log(df['close'] / df['close'].shift(1))
df['realized_vol'] = 100 * np.sqrt(365 / window_size * df['log_return'].rolling(window_size).std())

# Compute annualized realized volatility Garman Klass model with rolling window
df['gamma'] = 0.5 * np.log(df['high'] / df['low']) ** 2 - (2 * np.log(2) - 1) * np.log(
    df['close'] / df['open']) ** 2
df['gk_vol'] = 100 * np.sqrt(df['gamma'].rolling(window_size).sum() / (window_size - 1)) * np.sqrt(365)

df.dropna(inplace=True)
# Compute the annualized GARCH volatility using a rolling window
garch_model = arch.arch_model(df['log_return'], vol='GARCH', p=1, q=1)
garch_volatility = garch_model.fit(disp='off').conditional_volatility
df['garch_vol'] = 100 * garch_volatility * np.sqrt(365)

# Compute the annualized HAR volatility with a rolling window
df['volatility_5d'] = df['log_return'].rolling(5).std() * np.sqrt(365)
df['volatility_21d'] = df['log_return'].rolling(21).std() * np.sqrt(365)
df['volatility_63d'] = df['log_return'].rolling(63).std() * np.sqrt(365)
df['har_vol'] = 100 * np.sqrt(0.5 * (0.5 * df['volatility_5d']**2 + 0.3 * df['volatility_21d']**2 + 0.2 * df['volatility_63d']**2))

# Compute rolling window annualized volatility using EGARCH model
egarch_vol = arch.arch_model(df['log_return'], vol='egarch', p=1, o=1, q=1, dist='Normal').fit(disp='off').conditional_volatility
df['egarch_vol'] = 100 * np.sqrt(365) * egarch_vol.rolling(window=window_size).mean()

# Compute rolling window annualized volatility using GJR-GARCH model
gjr_vol = arch.arch_model(df['log_return'], vol='garch', p=1, o=1, q=1, dist='Normal', power=1.0).fit(disp='off').conditional_volatility
df['gjr_vol'] = 100 * np.sqrt(365) * gjr_vol.rolling(window=window_size).mean()

df.dropna(inplace=True)


/opt/anaconda3/lib/python3.9/site-packages/arch/univariate/base.py:310: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001306. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


/opt/anaconda3/lib/python3.9/site-packages/arch/univariate/base.py:310: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001306. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


/opt/anaconda3/lib/python3.9/site-packages/arch/univariate/base.py:310: DataScaleWarning:

y is poorly scaled,

In [132]:
df.head()

,open,high,low,close,volume,log_return,realized_vol,gamma,gk_vol,garch_vol,volatility_5d,volatility_21d,volatility_63d,har_vol,egarch_vol,gjr_vol
timestamp,,,,,,,,,,,,,,,,
2020-11-18,17659.38,18476.93,17214.45,17776.12,149019.788134,0.006589,60.468501,0.002488,67.951276,66.533313,0.607836,0.596766,0.470291,40.975952,55.103300,55.080363
2020-11-19,17777.75,18179.80,17335.65,17802.82,93009.561008,0.001501,60.552759,0.001130,68.629487,64.790894,0.538312,0.599008,0.470026,38.517398,55.594284,55.533914
2020-11-20,17802.81,18815.22,17740.04,18655.67,88423.018489,0.046793,59.263065,0.000885,65.992482,63.121592,0.480314,0.614285,0.478842,37.041356,55.995707,55.894400
2020-11-21,18655.66,18965.90,18308.58,18703.80,75577.458394,0.002577,59.319363,0.000620,65.928716,64.987697,0.500069,0.616458,0.478897,37.743143,56.259095,56.128620
2020-11-22,18703.80,18750.00,17610.86,18414.43,81645.737778,-0.015592,59.635269,0.001870,67.429052,63.307596,0.441326,0.625572,0.479189,36.104745,56.453963,56.293439


In [134]:
fig = px.line(df, x=df.index, y=['realized_vol', 'gk_vol', 'har_vol', 'egarch_vol', 'gjr_vol'],
              title='Bitcoin Annualized Realized Volatility',
              labels={'value': 'Volatility', 'variable': 'Method'})
fig.update_xaxes(title='Date')
# fig.update_yaxes(tickformat=)
fig.show()